In [43]:
# Libraries setup
import os
import warnings
import pandas as pd
import numpy as np

from dotenv import load_dotenv
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm import tqdm

warnings.filterwarnings('ignore')
load_dotenv()

True

### Data Load Pipeline
The data load pipeline is almost the same as in EDA. For the purpose of demonstration, it is not a new module, but placed as a function.

In [44]:
# Define paths
fpath_data = os.environ.get("FPATH_DATA")
fpath_dicts = os.environ.get("FPATH_DICTS")

def get_data_order_items():
    # Load order items data
    order_items = pd.read_csv(fpath_data+"order_items.csv")
    # Convert shipping_limit_date to timestamps
    order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date'])
    # Lets load data regarding product category, drop redundant columns, check the data, and then join the data to order_items
    product_categories = pd.read_csv(fpath_data+"products.csv")
    product_categories = product_categories[["product_id", "product_category_name"]]
    # Drop nulls
    product_categories = product_categories.dropna()
    # Map categories to english ones
    categories_translation_data = pd.read_csv(fpath_data+"product_category_name_translation.csv")
    product_categories = pd.merge(product_categories, categories_translation_data, how='left', on='product_category_name')
    # Instead of dropping 13 rows w/o translation, use the same data as originally
    product_categories['product_category_name_english'] = product_categories['product_category_name_english'].fillna(product_categories['product_category_name'])
    # Join product category to orders
    order_items = pd.merge(order_items, product_categories, how='left', on='product_id')
    # I decided to drop those without any category prescribed as those might be incorrect data
    order_items = order_items.dropna(subset=['product_category_name'])
    # Join data regarding purchase timestamp - first, read data
    orders = pd.read_csv(fpath_data+'orders.csv')
    # Keep only relevant columns
    orders = orders[['order_id','order_purchase_timestamp']]
    # Check for nulls
    orders.isnull().sum()
    # Transform order_purchase_timestamp to a timestamp object
    orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
    # Join orders data via inner - we could do also left join, but it would need to double check for nulls
    order_items = pd.merge(order_items, orders, how='inner', on='order_id')
    # Drop redundant data, not related to this task
    # As this is just a demonstration, we won't be adding information such as holidays for a different set of countries the customers are from etc
    order_items = order_items.drop(columns=['freight_value','shipping_limit_date','product_category_name'])
    # Add a set of information regarding sale timestamp
    order_items['Purchase Date'] = pd.to_datetime(order_items['order_purchase_timestamp']).dt.floor('D')
    return order_items

In [45]:
order_items = get_data_order_items()

dataset = order_items.groupby(['product_category_name_english', 'Purchase Date']).agg(
    sold_products_quantity=('order_item_id', 'size')
    # sellers_count=('seller_id', 'count')
).sort_values(by=['product_category_name_english', 'Purchase Date'], ascending=True).reset_index()
dataset

,product_category_name_english,Purchase Date,sold_products_quantity
0,agro_industry_and_commerce,2017-01-23,2
1,agro_industry_and_commerce,2017-01-31,1
2,agro_industry_and_commerce,2017-02-05,1
3,agro_industry_and_commerce,2017-02-08,1
4,agro_industry_and_commerce,2017-02-12,1
...,...,...,...
18499,watches_gifts,2018-08-25,3
18500,watches_gifts,2018-08-26,2
18501,watches_gifts,2018-08-27,2
18502,watches_gifts,2018-08-28,1


### Features Engineering
Feature engineering is the process of creating new features or transforming existing ones to improve 
the performance of machine learning models. 

##### Definition of new features functions

In [46]:
# Time Based Features
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_ = X.copy()
        X_['dayofweek'] = X_.index.dayofweek
        X_['quarter'] = X_.index.quarter
        X_['month'] = X_.index.month
        X_['year'] = X_.index.year
        X_['dayofyear'] = X_.index.dayofyear
        X_['dayofmonth'] = X_.index.day
        X_['weekofyear'] = X_.index.isocalendar().week
        X_['product_category_name_english'] = X_['product_category_name_english'].astype('category')
        return X_

# # Lag Features
# def create_lag_features(df, column, lags):
#     for lag in lags:
#         df[f'{column}_lag_{lag}'] = df.groupby('product_category_name_english')[column].shift(lag)
#     return df

# # Rolling Statistics
# def create_rolling_features(df, column, windows):
#     for window in windows:
#         df[f'{column}_rolling_mean_{window}'] = df.groupby('product_category_name_english')[column].rolling(window=window).mean().reset_index(0,drop=True)
#         df[f'{column}_rolling_std_{window}'] = df.groupby('product_category_name_english')[column].rolling(window=window).std().reset_index(0,drop=True)
#     return df

# # Exponential Moving Average
# def create_ema_features(df, column, spans):
#     for span in spans:
#         df[f'{column}_ema_{span}'] = df.groupby('product_category_name_english')[column].ewm(span=span).mean().reset_index(0,drop=True)
#     return df

# # Fourier Terms for Seasonality
# def fourier_series(dates, period, order):
#     t = (dates - pd.Timestamp("1970-01-01")) / pd.Timedelta('1D')
#     return pd.DataFrame({f'fourier_cos_{period}_{n}': np.cos(2 * n * np.pi * t / period),
#                          f'fourier_sin_{period}_{n}': np.sin(2 * n * np.pi * t / period)}
#                         for n in range(1, order + 1))

# # Price-related Features
# def price_related_features(df):
#     df['price_rolling_mean'] = df.groupby('product_category_name_english')['price'].rolling(window=7).mean().reset_index(0,drop=True)
#     df['price_relative_to_mean'] = df['price'] / df['price_rolling_mean']
#     return df

# # Sold Items Quantity Features
# def quantity_related_features(df):
#     # Sort the dataframe by timestamp and category
#     df = df.sort_values(['product_category_name_english', 'order_purchase_timestamp'])
#     # Calculate rolling count of sold items quantity
#     df['quantity_rolling_count'] = df.groupby('product_category_name_english')['order_item_id'].transform(
#         lambda x: x.rolling(window='7D').count()
#     )
#     # Calculate daily quantity
#     df['daily_quantity'] = df.groupby(['product_category_name_english', df['order_purchase_timestamp'].dt.date])['order_item_id'].transform('count')
#     # Calculate quantity relative to rolling count
#     df['quantity_relative_to_count'] = df['daily_quantity'] / df['quantity_rolling_count']
#     # Calculate cumulative sum of sold items within each category
#     df['cumulative_quantity'] = df.groupby('product_category_name_english')['daily_quantity'].cumsum()
#     return df

##### Final data pipeline that leverages functions written above

In [47]:
def prepare_data(dataset):
    dataset['Purchase Date'] = pd.to_datetime(dataset['Purchase Date'])
    daily_sales = dataset.groupby(['Purchase Date', 'product_category_name_english'])['sold_products_quantity'].sum().reset_index()
    return daily_sales.sort_values(['product_category_name_english', 'Purchase Date'])

def create_forecast_data(last_date, forecast_end, category):
    forecast_start = last_date + pd.Timedelta(days=1)
    forecast_dates = pd.date_range(start=forecast_start, end=forecast_end, freq='D')
    forecast_data = pd.DataFrame({'Purchase Date': forecast_dates, 'product_category_name_english': category})
    forecast_data.set_index('Purchase Date', inplace=True)
    return forecast_data

##### Evaluation metrics

In [48]:
def wmae(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred) * np.arange(1, len(y_true)+1)) / np.sum(np.arange(1, len(y_true)+1))

def bias(y_true, y_pred):
    return np.mean(y_pred - y_true)

##### Main function

In [49]:
def xgboost_forecast(data, category, forecast_end='2018-09-24'):
    data = data.set_index('Purchase Date')
    
    # Define features
    features = ['product_category_name_english', 'dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'weekofyear']
    
    # Create pipeline
    pipeline = Pipeline([
        ('feature_engineer', FeatureEngineer()),
        ('xgb', XGBRegressor(n_estimators=1000, learning_rate=0.05, random_state=42, enable_categorical=True))
    ])
    
    # Split data into train and test
    split_date = data.index.max() - pd.Timedelta(days=30)  # Use last 30 days as test set
    train_data = data[data.index <= split_date]
    test_data = data[data.index > split_date]
    
    # Fit the pipeline
    pipeline.fit(train_data[['product_category_name_english']], train_data['sold_products_quantity'])
    
    # Make in-sample predictions
    test_data_transformed = pipeline.named_steps['feature_engineer'].transform(test_data[['product_category_name_english']])
    in_sample_forecast = pipeline.named_steps['xgb'].predict(test_data_transformed[features])
    
    # Prepare and make out-of-sample forecast
    last_date = data.index.max()
    forecast_data = create_forecast_data(last_date, forecast_end, category)
    forecast_data_transformed = pipeline.named_steps['feature_engineer'].transform(forecast_data)
    out_of_sample_forecast = pipeline.named_steps['xgb'].predict(forecast_data_transformed[features])
    forecast_data['forecast'] = out_of_sample_forecast
    
    return test_data, in_sample_forecast, forecast_data

##### Main execution

In [50]:
data = prepare_data(dataset)

forecasts = {}
metrics = {}

for category in tqdm(data['product_category_name_english'].unique(), desc="Processing categories"):
    category_data = data[data['product_category_name_english'] == category]
    
    # Check last available date for the category
    last_date = category_data['Purchase Date'].max()
    print(f"\nCategory: {category}, Last date: {last_date}")
    
    if last_date >= pd.Timestamp('2018-09-24'):
        print(f"Skipping forecast for {category} as data is available up to or beyond 2018-09-24")
        continue
    
    test_data, in_sample_forecast, forecast_data = xgboost_forecast(category_data, category, forecast_end='2018-09-24')
    
    # Calculate metrics
    mae = mean_absolute_error(test_data['sold_products_quantity'], in_sample_forecast)
    wmae_score = wmae(test_data['sold_products_quantity'].values, in_sample_forecast)
    bias_score = bias(test_data['sold_products_quantity'].values, in_sample_forecast)
    
    metrics[category] = {
        'MAE': mae,
        'WMAE': wmae_score,
        'Bias': bias_score,
        'Last_Date': last_date
    }
    
    forecasts[category] = forecast_data['forecast']

Processing categories:   0%|          | 0/73 [00:00<?, ?it/s]


Category: agro_industry_and_commerce, Last date: 2018-08-26 00:00:00


Processing categories:   1%|▏         | 1/73 [00:00<00:26,  2.76it/s]


Category: air_conditioning, Last date: 2018-08-25 00:00:00


Processing categories:   3%|▎         | 2/73 [00:00<00:30,  2.34it/s]

...

Processing categories:  97%|█████████▋| 71/73 [00:33<00:01,  1.93it/s]


Category: toys, Last date: 2018-08-29 00:00:00


Processing categories:  99%|█████████▊| 72/73 [00:33<00:00,  1.70it/s]


Category: watches_gifts, Last date: 2018-08-29 00:00:00


Processing categories: 100%|██████████| 73/73 [00:34<00:00,  2.11it/s]


In [51]:
# Create DataFrames with all forecasts and metrics
forecast_df = pd.DataFrame(forecasts)
metrics_df = pd.DataFrame(metrics).T

print("\nForecasts up to 2018-09-24:")
print(forecast_df)
print("\nMetrics for each category:")
print(metrics_df)

forecast_df.to_csv('XGBoost_category_demand_forecasts.csv')
metrics_df.to_csv('XGBoost_category_demand_metrics.csv')
print("Forecasts have been saved to 'XGBoost_category_demand_forecasts.csv'")
print("Metrics have been saved to 'XGBoost_category_demand_metrics.csv'")


Forecasts up to 2018-09-24:
               agro_industry_and_commerce  air_conditioning       art  \
Purchase Date                                                           
2017-09-18                            NaN               NaN       NaN   
2017-09-19                            NaN               NaN       NaN   
2017-09-20                            NaN               NaN       NaN   
2017-09-21                            NaN               NaN       NaN   
2017-09-22                            NaN               NaN       NaN   
...                                   ...               ...       ...   
2018-09-20                       2.094255          1.039503  1.002913   
2018-09-21                       2.508539          1.139947  0.978940   
2018-09-22                       1.822733          0.958454  1.105342   
2018-09-23                       2.015668          1.043854  1.122530   
2018-09-24                       1.655123          1.137339  1.130967   

               arts_a